10分足を1か月ごとに取得するコード
df形式に変換する
CSVファイルに出力する

In [1]:
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
from dateutil.relativedelta import relativedelta

In [2]:
# API接続設定のファイルを読み込む
import configparser

# 設定
config = configparser.ConfigParser()
config.read('./config_v1.txt') # パスの指定が必要です
account_ID = config['oanda']['account_id']
api_key = config['oanda']['api_key']

In [3]:
api = API(access_token=api_key, environment="practice")

In [4]:
#candleデータを取得する処理をメソッドにしました。
#fromとtoをdatetime型で受けて、Oanda APIでcandleデータを取得します。
#datetimeのisoformat()で、Oanda APIで利用できるisoフォーマットに変換する必要があります。

# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

In [5]:
#Oanda APIからの戻り値はJSON形式なので、
#それをpythonの配列に変換するメソッドを用意しました。
#戻り値の日付をisoフォーマットからdatatime形式へ変換していますが、
#小数点以下があるとうまく変換できなかったので、
#秒までの頭19桁で切ったものをdatetimeに変換する事にしました。

def oandaJsonToPythonList(JSONRes):
    data = []
    for res in JSONRes['candles']:
        data.append( [
            datetime.datetime.fromisoformat(res['time'][:19]),
            res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
    return data

In [6]:
#Oanda APIで価格データを取得するには、一度に5,000件までという制約があります。
#10分足だと1日分が6 * 24 = 144件、約34日分が5000件となります。
#そのため、1ヶ月分づつをループ処理で回しながら取得していくのが良いでしょう。

year_months =[
    [2016, 1], [2016, 2], [2016, 3], [2016, 4], [2016, 5], [2016, 6], [2016, 7], [2016, 8], [2016, 9], [2016, 10], [2016, 11], [2016, 12],
    [2017, 1], [2017, 2], [2017, 3], [2017, 4], [2017, 5], [2017, 6], [2017, 7], [2017, 8], [2017, 9], [2017, 10], [2017, 11], [2017, 12],
    [2018, 1], [2018, 2], [2018, 3], [2018, 4], [2018, 5], [2018, 6], [2018, 7], [2018, 8], [2018, 9], [2018, 10], [2018, 11], [2018, 12],
    [2019, 1], [2019, 2], [2019, 3], [2019, 4], [2019, 5], [2019, 6], [2019, 7], [2019, 8], [2019, 9], [2019, 10], [2019, 11], [2019, 12],
    [2020, 1], [2020, 2], [2020, 3], [2020, 4], [2020, 5], [2020, 6], [2020, 7], [2020, 8], [2020, 9], [2020, 10], [2020, 11]
]

In [7]:
#時間はUTCなので注意
#ヘルパーメソッドが揃ったところで、年月でループしながらデータを取得していきます
#上記で定義したメソッドを呼び出して配列に格納していくだけです。
#M10は10分データを意味します。

all_data = []
# year, monthでループ
for year, month in year_months:
    date_from = datetime.datetime(year, month, 1)
    date_to = date_from + relativedelta(months=+1, day=1)

    ret = getCandleDataFromOanda("USD_JPY", api, date_from, date_to, "M10")
    month_data = oandaJsonToPythonList(ret)

    all_data.extend(month_data)

In [8]:
# pandas DataFrameへ変換
df = pd.DataFrame(all_data)
df.columns = ['Datetime', 'Volume', 'Open', 'High', 'Low', 'Close']
df = df.set_index('Datetime')

In [9]:
df.shape

(183228, 5)

In [10]:
df.head(5)

,Volume,Open,High,Low,Close
Datetime,,,,,
2016-01-03 22:00:00,162,120.195,120.235,120.194,120.227
2016-01-03 22:10:00,208,120.226,120.253,120.209,120.236
2016-01-03 22:20:00,333,120.235,120.283,120.233,120.274
2016-01-03 22:30:00,359,120.274,120.304,120.268,120.286
2016-01-03 22:40:00,242,120.288,120.330,120.277,120.313


In [11]:
df.tail(5)

,Volume,Open,High,Low,Close
Datetime,,,,,
2020-12-01 04:10:00,137,104.440,104.446,104.426,104.426
2020-12-01 04:20:00,175,104.426,104.427,104.388,104.392
2020-12-01 04:30:00,312,104.390,104.394,104.373,104.378
2020-12-01 04:40:00,214,104.378,104.388,104.378,104.386
2020-12-01 04:50:00,170,104.388,104.392,104.362,104.376


In [12]:
# CSVファイルへの出力
df.to_csv('./USD_JPY_201601-202011_M10.csv')

# CSVファイルの読み取り
df = pd.read_csv('./USD_JPY_201601-202011_M10.csv', index_col='Datetime')

PermissionError: [Errno 13] Permission denied: './USD_JPY_201601-202011_M10.csv'